In [2]:
pip install petastorm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 602.4 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 9.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached pyspark-3.4.1-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492035 sha256=6773641352a8ff5b4d135a7c399067030207253a06294906304ee2dbc6ffd6da
  Stored in directory: /home/shreyasve/.cache/pip/wheels/bf/5d/6a/2e53874f7ec4e2bede522385439531fafec8fafe005b5c3d1b
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from pyarrow.parquet import ParquetFile
import pyarrow.parquet as pq
# import s3fs
# s3_filesystem = s3fs.S3FileSystem()
import glob
import pandas as pd
import numpy as np
import boto3
import ast
import time
import numpy as np
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from random import sample
from tqdm import tqdm
import petastorm
from petastorm.pytorch import DataLoader
from petastorm import make_reader
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")
import time
import logging

ModuleNotFoundError: No module named 'petastorm'

In [2]:
# SAMPLING_RATIO = 3 # BATCH_SIZE_NEG : BATCH_SIZE_POS 
# BATCH_SIZE_POS = 16
# BATCH_SIZE_NEG = BATCH_SIZE_POS * SAMPLING_RATIO
os.chdir(os.path.join(os.path.expanduser("~"),'ItemSage_Data_v3'))
home_dir = os.getcwd()
TRAIN_PATH = 'train3_img_flattened'
# TEST_PATH = 'test_img_flattened'
VAL_PATH = 'val3_img_flattened'
TRAIN_SIZE = 1757020
NUM_EPOCHS = 25
ENGAGEMENT_TYPE = 'all' # clicks/saves/all

LOG_INTERVAL = 1300 # just to keep ~10 reporting logs per epoch
BATCH_SIZE = 128

HOME_FEATS_DIM = 30
IMAGE_INPUT_EMBED_DIM = 512 
NUM_IMAGES = 10
TEXT_INPUT_EMBED_DIM = 1536 
SKG_INPUT_EMBED_DIM = 32  # skip-gram
PS_INPUT_EMBED_DIM = 128 # pinsage
TRANSFORMER_INPUT_DIM = 512
ITEMSAGE_EMBED_DIM = 256
N_ZIP_CATEGORIES = 14 # look this up everytime while creating data
ZIP_EMBED_DIM = 16

LOSS_TYPE = "contrastive" # contrastive/classification/itemsage_loss
# LOSS_FUNC = nn.BCELoss # classification : BCELoss() | contrastive : nn.CosineEmbeddingLoss() | itemsage_loss : simCLRLoss() [custom - to be implemented]
CONTRASTIVE_LOSS_MARGIN = 0.1
DEVICE = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)


In [3]:
model_args = {
    'n_epochs' : NUM_EPOCHS,
    'log_interval' : LOG_INTERVAL,
    'bsz':BATCH_SIZE,
    'engagement_type' : ENGAGEMENT_TYPE,
    'loss_type': LOSS_TYPE,
    'loss' : nn.BCELoss if LOSS_TYPE == 'classification' else nn.CosineEmbeddingLoss,
    'loss_margin':None if LOSS_TYPE == 'classification' else CONTRASTIVE_LOSS_MARGIN,
    'home_features_dim' : HOME_FEATS_DIM,
    'image_embed_dim' : IMAGE_INPUT_EMBED_DIM,
    'n_img' : NUM_IMAGES,\
    'text_embed_dim' : TEXT_INPUT_EMBED_DIM,
    'skg_embed_dim' : SKG_INPUT_EMBED_DIM,
    'ps_embed_dim' : PS_INPUT_EMBED_DIM,
    'transformer_input_dim' : TRANSFORMER_INPUT_DIM,
    'final_embed_dim' : ITEMSAGE_EMBED_DIM,
    'n_zips':N_ZIP_CATEGORIES,
    'zip_embed_dim' : ZIP_EMBED_DIM,
    'device' : DEVICE,
    'train_path':''.join(['file://',os.path.join(home_dir, TRAIN_PATH)]),
    # 'test_path':''.join(['file://',os.path.join(home_dir, TEST_PATH)]),
    'val_path':''.join(['file://',os.path.join(home_dir, VAL_PATH)]),
    'model_path': os.path.join(home_dir, 'Models'),
    'train_size' : TRAIN_SIZE
}

cuda_args = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

model_args = {**model_args, **cuda_args}

model_save_dir = Path(os.path.join(model_args['model_path'],'_'.join(['model',LOSS_TYPE,datetime.now().strftime('%Y-%m-%d_%H-%M-%S')])))
if not model_save_dir.exists():
        model_save_dir.mkdir(parents=True)
        
log_file = os.path.join(model_save_dir,'output.log')
logging.basicConfig(filename=log_file, level=logging.INFO)



In [4]:
## TODO : 1. Add Positional Encoding  - the one in Attention paper [sinusoidal,static]
#                                     - as a trainable encoding (?) - check this out, seems interesting
# dont know how much it would make sense - lets try it out and see if there is any diff in perf

# SIAMESE NETWORK IMPLEMENTATION FOR ITEMSAGE

class ItemSageModel(nn.Module):
    
    def __init__(self, nhead = 8,nlayers = 1,**kwargs):
        super(ItemSageModel,self).__init__()
        
        home_feat_dim = kwargs['home_features_dim']
        image_embedding_dim = kwargs['image_embed_dim']
        text_embedding_dim = kwargs['text_embed_dim']
        skg_embedding_dim = kwargs['skg_embed_dim']
        ps_embedding_dim = kwargs['ps_embed_dim']
        transformer_input_dim = kwargs['transformer_input_dim']
        output_dim = kwargs['final_embed_dim']
        n_zips = kwargs['n_zips']
        zip_embed_dim = kwargs['zip_embed_dim']
        
        if kwargs['loss_margin'] :
            self.loss = kwargs['loss'](margin=kwargs['loss_margin']) # criterion created
        else:
            self.loss = kwargs['loss']( )
            
        self.device = kwargs['device']
        
        # converting all input to the same dimensionality for forming transformer input seq
        self.zip_embed = nn.Embedding(n_zips,zip_embed_dim)
        self.home_linear = nn.Linear(home_feat_dim+zip_embed_dim, transformer_input_dim)
        self.image_linear = nn.Linear(image_embedding_dim, transformer_input_dim)
        self.text_linear = nn.Linear(text_embedding_dim, transformer_input_dim)
        self.skg_linear = nn.Linear(skg_embedding_dim, transformer_input_dim)
        self.ps_linear = nn.Linear(ps_embedding_dim, transformer_input_dim)
        
        # transforming the CLS token
        self.global_CLS_linear = nn.Linear(1, transformer_input_dim)
        
        # Transformer Encoder Layer
        self.encoder_layer = TransformerEncoderLayer(d_model=transformer_input_dim, nhead=nhead,\
                                                     batch_first=True,norm_first=True)
        self.transformer_encoder = TransformerEncoder(self.encoder_layer, num_layers=nlayers)
        
        # MLP head
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(transformer_input_dim),
            nn.Linear(transformer_input_dim, 1024),
            nn.GELU(),
            nn.Linear(1024, output_dim)
        )
    
    # option to use one of the two loss types
    def _contrastive_loss(self, x1,x2,label):
        return self.loss(x1,x2,label)
    
    def _classification_loss(self,score,label):
        return self.loss(score,label)

    # itemsage tower
    def _encoder(self, text_embeddings, image_embeddings, skg_embeddings, ps_embeddings, home_features, zip_idx):
        """ 
        :param image_embeddings: (bsz, seq_length, D_img) - 
        :param text_embeddings: (bsz,  D_text) - make seq_length = 1 for openAI embeddings - using unsqueeze()
        :param skg_embeddings: (bsz, D_skg) - make seq_length = 1 because 1 embedding/listing - using unsqueeze()
        :param ps_embeddings: (bsz, D_ps) - make seq_length = 1 because 1 embedding/listing - using unsqueeze()
        :param home_features: (bsz, D_home_feat) - make seq_length = 1 because 1 embedding/listing - using unsqueeze()
        """
        transformed_image_embeddings = self.image_linear(image_embeddings)
        transformed_text_embeddings = self.text_linear(torch.unsqueeze(text_embeddings,1))
        transformed_skg_embeddings = self.skg_linear(torch.unsqueeze(skg_embeddings,1))
        transformed_ps_embeddings = self.ps_linear(torch.unsqueeze(ps_embeddings,1))
        zip_embedding = self.zip_embed(zip_idx)
        home_features = torch.cat((home_features,zip_embedding),dim=1)
        transformed_home_feat = self.home_linear(torch.unsqueeze(home_features,1))
        
        # Apply linear transformation to the global token
        batch_size = transformed_image_embeddings.size(0)
        cls_token = torch.ones(batch_size, 1,1).to(self.device)

        
        # Apply linear transformation to make [CLS] token 512-dimensional
        cls_token = self.global_CLS_linear(cls_token)
        
        # print(cls_token.shape, transformed_image_embeddings.shape, transformed_text_embeddings.shape)
        # embedding sequence
        embedding_seq = torch.cat((cls_token, transformed_image_embeddings, transformed_text_embeddings,\
                                   transformed_skg_embeddings,transformed_ps_embeddings,transformed_home_feat), dim=1)
        # print('embedding_seq shape : ',embedding_seq.shape)
        transformer_output = self.transformer_encoder(embedding_seq)
        # print('transformer_output shape : ',transformer_output.shape)
        cls_output = transformer_output[:, 0, :]
        product_embedding = self.mlp_head(cls_output)
        return product_embedding
    
    
    def forward(self,**kwargs1):
        anchor_encoder = self._encoder(kwargs1['anchor'][0],kwargs1['anchor'][1],kwargs1['anchor'][2],\
                                      kwargs1['anchor'][3],kwargs1['anchor'][4],kwargs1['anchor'][5]) # query tower
        
        engaged_encoder = self._encoder(kwargs1['engaged'][0],kwargs1['engaged'][1],kwargs1['engaged'][2],\
                                       kwargs1['engaged'][3],kwargs1['engaged'][4],kwargs1['engaged'][5]) # example tower
        
        # normalize both embeddings so that cosine_similarity == dot_product
        return F.normalize(anchor_encoder,dim=1),F.normalize(engaged_encoder,dim=1)
    
    
class Metric:
    def __init__(self):
        pass

    def __call__(self, outputs, target, loss):
        raise NotImplementedError

    def reset(self):
        raise NotImplementedError

    def value(self):
        raise NotImplementedError

    def name(self):
        raise NotImplementedError
        

class AccumulatedAccuracyMetric(Metric):
    """
    Works with classification model
    """

    def __init__(self,margin=0.):
        self.correct = 0
        self.total = 0
        self.margin = margin

    def __call__(self, q_emb,p_emb, labels):
        
        labels = torch.where (labels == -1,0,labels)
        if self.margin:
            predicted_prob = F.sigmoid(F.cosine_similarity(q_emb,p_emb) - self.margin)
        else:
            predicted_prob = F.sigmoid(F.cosine_similarity(q_emb,p_emb))
        pred = torch.where (predicted_prob >=0.5,1,0)
 
        self.correct += pred.eq(labels).cpu().sum()
        self.total += len(labels)
        return self.value()
    
    def reset(self):
        self.correct = 0
        self.total = 0

    def value(self):
        return 100 * float(self.correct) / self.total

    def name(self):
        return 'Accuracy'
    
class AccumulatedF1Metric(Metric):
    """
    Works with classification model
    """

    def __init__(self,margin=0.):
        self.tp,self.fn,self.fp,self.tn = 0,0,0,0
        # self.total = 0
        self.margin = margin

    def __call__(self, q_emb,p_emb, labels):
        
        labels = torch.where (labels == -1,0,labels)
        if self.margin:
            predicted_prob = F.sigmoid(F.cosine_similarity(q_emb,p_emb) - self.margin)
            # pred = torch.where (predicted_prob >=0.5,1,-1)
        else:
            predicted_prob = F.sigmoid(F.cosine_similarity(q_emb,p_emb))
            
        pred = torch.where (predicted_prob >=0.5,1,0)
        confusion_vector = pred / labels
        self.tp +=  torch.sum(confusion_vector == 1).item()
        self.fp +=  torch.sum(confusion_vector == float('inf')).item()
        self.tn +=  torch.sum(torch.isnan(confusion_vector)).item()
        self.fn +=  torch.sum(confusion_vector == 0).item()

        return self.value()
    
    def reset(self):
        self.tp,self.fn,self.fp,self.tn = 0,0,0,0

    def value(self):
        precision = self.tp/(self.tp+self.fp)
        recall = self.tp/(self.tp+self.fn)
        return ( 2*precision*recall ) / ( recall + precision )

    def name(self):
        return 'F1-score'

In [5]:
def train_one_epoch(epoch,train_loader,model,optimizer,**kwargs):
    # model = kwargs['model']
    model.train()
    
    log_interval = kwargs['log_interval']
    # bsz = kwargs['bsz']
    device = kwargs['device']
    loss_type = kwargs['loss_type']
    metrics = kwargs['metrics']
    len_train_data = kwargs['train_size']
    for metric in metrics:
        metric.reset()
    train_loss = 0.
    losses = []
    
    # progress_bar = tqdm(total=BATCH_SIZE_POS, desc='Training Progress')
    
    counter = 0
    for batch_idx, batch in enumerate(train_loader):
        kwargs1 = {'anchor':[],'engaged':[]}
        
        #labels
        batch_labels = batch['label'].to(device)
        bsz = len(batch_labels)
        
        ## anchor tower input
        kwargs1['anchor']= [batch['text_embed1'].to(device),\
                            batch['img_embed1'].view(bsz,kwargs['n_img'],kwargs['image_embed_dim']).to(device),\
                            batch['skg_embed1'].to(device),\
                            batch['ps_embed1'].to(device),\
                            batch['home_features1'][:,:kwargs['home_features_dim']].to(device),\
                            batch['home_features1'][:,kwargs['home_features_dim']].to(torch.long).to(device)]
        ## engaged tower input
        kwargs1['engaged']= [batch['text_embed2'].to(device),\
                             batch['img_embed2'].view(bsz,kwargs['n_img'],kwargs['image_embed_dim']).to(device),\
                             batch['skg_embed2'].to(device),\
                             batch['ps_embed2'].to(device),\
                             batch['home_features2'][:,:kwargs['home_features_dim']].to(device),\
                             batch['home_features2'][:,kwargs['home_features_dim']].to(torch.long).to(device)]
        # model forward pass
        q_emb,p_emb = model(**kwargs1)
        
        # optimizer 
        optimizer.zero_grad()
        
        if loss_type ==  "contrastive" : # contrastive
            batch_labels = torch.where(batch_labels == 0,-1,batch_labels) # for CosineEmbeddingloss()
            loss = model._contrastive_loss( q_emb,p_emb,batch_labels )
        else: #classification
            scores = F.sigmoid( F.cosine_similarity( q_emb,p_emb ) ) # applying sigmoid to get score between 0 and 1
            loss = model._classification_loss( scores,batch_labels.to(torch.float32)  )
            
        # model backward pass
        loss.backward()
        # wt updation
        optimizer.step()
        train_loss += loss.item()
        losses.append(loss.item())
        counter += bsz
        
        for metric in metrics:
            metric(q_emb,p_emb, batch_labels)
        
        if batch_idx % log_interval == 0:
            message = 'Train : {} [{}/{} ({:.0f}%)]\tLoss: {:.4f}'.format(
                epoch, counter,len_train_data,100.*counter/len_train_data, np.round(np.mean(losses),4)) # np.round(train_loss/counter,4)
            for metric in metrics:
                message += '\t{}: {:.2f}'.format(metric.name(), metric.value())
            
            logging.info(message)
            print(message)
            losses = []
            
    train_loss /= (batch_idx+1)
    return train_loss,metrics # avg train epoch loss - averaged across all batches
    
def test_one_epoch(epoch,test_loader,model,optimizer,val=True,**kwargs):
    
    metrics = kwargs['metrics']
    with torch.no_grad():
        # metrics = kwargs['metrics']
        for metric in metrics:
            metric.reset()
            
    # model = kwargs['model']
    model.eval()
    
    log_interval = kwargs['log_interval']
    # bsz = kwargs['bsz']
    device = kwargs['device']
    loss_type = kwargs['loss_type']
    
    test_loss = 0
    correct = 0
    count = 0
    
    with torch.no_grad():
        for batch_idx,batch in enumerate(test_loader):
            kwargs1 = {'anchor':[],'engaged':[]}
            
            #labels
            batch_labels = batch['label'].to(device)
            bsz = len(batch_labels)
            
            ## anchor tower input
            kwargs1['anchor']= [batch['text_embed1'].to(device),\
                                batch['img_embed1'].view(bsz,kwargs['n_img'],kwargs['image_embed_dim']).to(device),\
                                batch['skg_embed1'].to(device),\
                                batch['ps_embed1'].to(device),\
                                batch['home_features1'][:,:kwargs['home_features_dim']].to(device),\
                                batch['home_features1'][:,kwargs['home_features_dim']].to(torch.long).to(device)]
            ## engaged tower input
            kwargs1['engaged']= [batch['text_embed2'].to(device),\
                                 batch['img_embed2'].view(bsz,kwargs['n_img'],kwargs['image_embed_dim']).to(device),\
                                 batch['skg_embed2'].to(device),\
                                 batch['ps_embed2'].to(device),\
                                 batch['home_features2'][:,:kwargs['home_features_dim']].to(device),\
                                 batch['home_features2'][:,kwargs['home_features_dim']].to(torch.long).to(device)]

            # model forward pass
            q_emb,p_emb = model(**kwargs1)
            
            
            if loss_type ==  "contrastive" : # contrastive
                batch_labels = torch.where(batch_labels == 0,-1,batch_labels) # for CosineEmbeddingloss()
                loss = model._contrastive_loss( q_emb,p_emb,batch_labels )
            else: #classification
                scores = F.sigmoid( F.cosine_similarity( q_emb,p_emb ) ) # applying sigmoid to get score between 0 and 1
                loss = model._classification_loss( scores,batch_labels.to(torch.float32) )
            
            test_loss += loss.item()  # sum up batch loss
            count += bsz
            for metric in metrics:
                metric(q_emb,p_emb, batch_labels)
    test_loss /= (batch_idx+1)
    return test_loss,metrics
    
    
    
    


In [9]:
ItemSageModel(**model_args)

ItemSageModel(
  (loss): CosineEmbeddingLoss()
  (zip_embed): Embedding(14, 16)
  (home_linear): Linear(in_features=46, out_features=512, bias=True)
  (image_linear): Linear(in_features=512, out_features=512, bias=True)
  (text_linear): Linear(in_features=1536, out_features=512, bias=True)
  (skg_linear): Linear(in_features=32, out_features=512, bias=True)
  (ps_linear): Linear(in_features=128, out_features=512, bias=True)
  (global_CLS_linear): Linear(in_features=1, out_features=512, bias=True)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
    )
    (linear1): Linear(in_features=512, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=512, bias=True)
    (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)

In [6]:
model = ItemSageModel(**model_args).to(model_args['device'])
model_args['metrics'] = [AccumulatedAccuracyMetric(model_args['loss_margin']),AccumulatedF1Metric(model_args['loss_margin'])]
lr = 1e-2
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)

early_stopping_patience = 5  # Number of epochs without improvement before stopping
best_validation_loss = float('inf')
epochs_without_improvement = 0
best_epoch = -1
best_accuracy = 0.
# model_save_dir = Path(os.path.join(model_args['model_path'],'_'.join(['model',datetime.now().strftime('%Y-%m-%d_%H-%M-%S')])))

if model_args['loss_margin'] :
    logging.info(" ###### USING CONTRASTIVE LOSS ######\n ")
    print( " ###### USING CONTRASTIVE LOSS ######\n ")
else :
    logging.info( " ###### USING CLASSIFICATION LOSS ######\n ")
    print( " ###### USING CLASSIFICATION LOSS ######\n ")
    
for epoch in range(1,model_args['n_epochs']+1 ):
    scheduler.step()
    tic = time.time()
    with DataLoader(make_reader(model_args['train_path'], num_epochs=1, seed=1, shuffle_rows=True),
                    batch_size=model_args['bsz']) as train_loader:
        epoch_train_loss,metrics = train_one_epoch(epoch,train_loader,model,optimizer,**model_args)
    message = '\nEpoch: {}/{}. Train set: Average loss: {:.4f}'.format(epoch , model_args['n_epochs'], epoch_train_loss)
    
    for metric in metrics:
        message += '\t{}: {:.2f}'.format(metric.name(), metric.value())
    
    with DataLoader(make_reader(model_args['val_path'], num_epochs=1, seed=1, shuffle_rows=True),
                    batch_size=model_args['bsz']) as val_loader:
        epoch_val_loss,metrics = test_one_epoch(epoch,val_loader,model,optimizer,**model_args)
    message += '\nEpoch: {}/{:.2f}. Validation set: Average loss: {:.4f}'.format(epoch , model_args['n_epochs'],epoch_val_loss)
    
    for metric in metrics:
        message += '\t{}: {:.2f}'.format(metric.name(), metric.value())
    
    # save epoch model states
    model_filename = 'model_state_epoch_{}.pth'.format(epoch)
    torch.save(model.state_dict(), os.path.join(model_save_dir,model_filename))
    toc = time.time()
    message += '\nEpoch time : {} min. Model state saved to : {}'.format(np.round((toc-tic)/60,2),os.path.join(model_save_dir,model_filename))
    
    logging.info(message)
    print(message)
    
    # Early Stopping based on train_loss - val_loss and val accuracy
    acc = metrics[0].value()
    # epoch_loss_diff = abs(epoch_train_loss - epoch_val_loss)
    # if (epoch_loss_diff < min_diff_till_now) & (epoch_val_acc >= best_val_acc-1):
    #     min_diff_till_now = epoch_loss_diff
    #     best_val_acc = epoch_val_acc
    #     epochs_without_improvement = 0
    # else:
    #     epochs_without_improvement += 1
        
    if acc > best_accuracy:
        best_accuracy = acc
        best_epoch = epoch
    elif epoch - best_epoch > early_stopping_patience:
        # print("Early stopped training at epoch %d" % epoch)
        logging.info("Early stopping triggered. No improvement in validation Accuracy. Best Epoch : {}".format(best_epoch))
        print("Early stopping triggered. No improvement in validation Accuracy. Best Epoch : {}".format(best_epoch))
        break  # terminate the training loop
        
    # if epoch_val_loss < best_validation_loss:
    #     best_validation_loss = epoch_val_loss
    #     epochs_without_improvement = 0
    # else:
    #     epochs_without_improvement += 1
    
    # if epochs_without_improvement >= early_stopping_patience:
    #     logging.info("Early stopping triggered. No improvement in validation loss.")
    #     print("Early stopping triggered. No improvement in validation loss.")
    #     break
    

 ###### USING CLASSIFICATION LOSS ######
 
Train : 0 [128/1757020 (0%)]	Loss: 0.9902	Accuracy: 30.47	F1-score: 0.47
Train : 0 [166528/1757020 (9%)]	Loss: 0.8181	Accuracy: 49.29	F1-score: 0.49
Train : 0 [332928/1757020 (19%)]	Loss: 0.7230	Accuracy: 52.96	F1-score: 0.49
Train : 0 [499328/1757020 (28%)]	Loss: 0.6688	Accuracy: 56.64	F1-score: 0.51
Train : 0 [665728/1757020 (38%)]	Loss: 0.6802	Accuracy: 58.14	F1-score: 0.51
Train : 0 [832128/1757020 (47%)]	Loss: 0.7245	Accuracy: 57.88	F1-score: 0.48
Train : 0 [998528/1757020 (57%)]	Loss: 0.7067	Accuracy: 58.26	F1-score: 0.47
Train : 0 [1164928/1757020 (66%)]	Loss: 0.6559	Accuracy: 59.64	F1-score: 0.49
Train : 0 [1331328/1757020 (76%)]	Loss: 0.6589	Accuracy: 60.33	F1-score: 0.51
Train : 0 [1497728/1757020 (85%)]	Loss: 0.6537	Accuracy: 61.10	F1-score: 0.51
Train : 0 [1664128/1757020 (95%)]	Loss: 0.6772	Accuracy: 61.44	F1-score: 0.51

Epoch: 1/25. Train set: Average loss: 0.6966	Accuracy: 61.45	F1-score: 0.51
Epoch: 1/25.00. Validation set: Av

In [1]:
!rm -rf Models

In [12]:
size = 1757020
size//128

13726

In [13]:
13726//10

1372